In [1]:
import os
import cv2
import numpy as np
from time import time
import matplotlib.pyplot as plt
import copy 
import random
from skimage import io
import math



Inspired by https://github.com/maitek/image_stacking (stolen)


In [3]:
# Align and stack images with ECC method
# Slower but more accurate
def stackImagesECC(file_list):
    M = np.eye(3, 3, dtype=np.float32)

    first_image = None
    stacked_image = None
    i = 0;
    for file in file_list:
        i += 1;
        image = cv2.imread(file,1).astype(np.float32) / 255
        print(file)
        if first_image is None:
            # convert to gray scale floating point image
            first_image = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
            stacked_image = image
            plt.imshow(cv2.cvtColor(stacked_image))
            
        else:
            # Estimate perspective transform
            s, M = cv2.findTransformECC(cv2.cvtColor(image,cv2.COLOR_BGR2GRAY), first_image, M, cv2.MOTION_HOMOGRAPHY)
            w, h, _ = image.shape
            # Align image to first image
            image = cv2.warpPerspective(image, M, (h, w))
            show = image.copy()
            show /= i
            show = (show*255).astype(np.uint8)
            stacked_image += image

    stacked_image /= len(file_list)
    stacked_image = (stacked_image*255).astype(np.uint8)
    return stacked_image



In [5]:
# Align and stack images by matching ORB keypoints
# Faster but less accurate
def stackImagesKeypointMatching(file_list):

    orb = cv2.ORB_create()

    # disable OpenCL to because of bug in ORB in OpenCV 3.1
    cv2.ocl.setUseOpenCL(False)

    stacked_image = None
    first_image = None
    first_kp = None
    first_des = None
    for file in file_list:
        print(file)
        image = cv2.imread(file,1)
        imageF = image.astype(np.float32) / 255

        # compute the descriptors with ORB
        kp = orb.detect(image, None)
        kp, des = orb.compute(image, kp)

        # create BFMatcher object
        matcher = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

        if first_image is None:
            # Save keypoints for first image
            stacked_image = imageF
            first_image = image
            first_kp = kp
            first_des = des
        else:
             # Find matches and sort them in the order of their distance
            matches = matcher.match(first_des, des)
            matches = sorted(matches, key=lambda x: x.distance)

            src_pts = np.float32(
                [first_kp[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
            dst_pts = np.float32(
                [kp[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)

            
             # Estimate perspective transformation
            M, mask = cv2.findHomography(dst_pts, src_pts, cv2.RANSAC, 5.0)

            w, h, _ = imageF.shape
            imageF = cv2.warpPerspective(imageF, M, (h, w))
            stacked_image += imageF
            plt.imshow(cv2.cvtColor(stacked_image))

    stacked_image /= len(file_list)
    stacked_image = (stacked_image*255).astype(np.uint8)
    return stacked_image

In [16]:
file_list = [];

print("loaded images")
for x in range(1,8):
  path = os.getcwd()+str(x)+".tif"
  file_list.append(path)
  print(path)
  
print("show images")
for i in range(len(file_list)):
  p = file_list[i]
  img = cv2.imread(p)
  cv2.imwrite('saved.tif', img)
  #plt.imshow(img)
  


loaded images
c:\Projects\P3-Project\Py image stacker1.tif
c:\Projects\P3-Project\Py image stacker2.tif
c:\Projects\P3-Project\Py image stacker3.tif
c:\Projects\P3-Project\Py image stacker4.tif
c:\Projects\P3-Project\Py image stacker5.tif
c:\Projects\P3-Project\Py image stacker6.tif
c:\Projects\P3-Project\Py image stacker7.tif
show images


error: OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\imgcodecs\src\loadsave.cpp:801: error: (-215:Assertion failed) !_img.empty() in function 'cv::imwrite'


In [8]:
print("stackImagesKeypointMatching")
img1 = stackImagesKeypointMatching(file_list)
#print("stackImagesECC")
#img2 = stackImagesECC(file_list)


stackImagesKeypointMatching


NameError: name 'file_list' is not defined

In [7]:
plt.imshow(cv2.cvtColor(img1))

NameError: name 'img1' is not defined